In [1]:
import pandas as pd
import numpy as np


In [2]:
df= pd.read_csv('smart_grid_data/File1.txt',delimiter=' ')

df.head()

,SMID,TimeID,Energy
0,1392,19503,0.140
1,1392,19504,0.138
2,1392,19505,0.140
3,1392,19506,0.145
4,1392,19507,0.145


In [3]:
id_list=df['SMID'].unique()
print(id_list)

[1392 1951 1491 1194 1804 1048 1802 1287 1529 1463 1860 1922 1334 1604
 1042 1494 1078 1984 1083 1978 1481 1524 1637 1783 1539 1777 1518 1698
 1094 1302 1824 1957 1020 1502 1625 1817 1196 1320 1105 1870 1224 1452
 1044 1920 1781 1139 1981 1340 1742 1549 1189 1362 1336 1106 1891 1182
 1532 1428 1264 1087 1014 1348 1948 1944 1073 1772 1077 1745 1592 1292
 1295 1941 1503 1008 1052 1835 1306 1823 1398 1528 1123 1852 1284 1297
 1773 1565 1590 1226 1419 1467 1173 1841 1966 1892 1969 1679 1462 1776
 1161 1190 1064 1215 1322 1238 1799 1207 1311 1765 1476 1757 1058 1548
 1596 1364 1415 1583 1780 1861 1806 1495 1380 1567 1588 1403 1651 1719
 1818 1225 1023 1466 1388 1522 1737 1324 1488 1178 1242 1411 1500 1644
 1180 1968 1343 1593 1068 1317 1489 1471 1971 1221 1949 1279 1004 1328
 1316 1646 1145 1381 1779 1027 1694 1630 1926 1154 1714 1352 1339 1516
 1026 1880 1541 1174 1365 1689 1416 1261 1110 1634 1063 1594 1599 1028
 1748 1176 1399 1149 1680 1276 1275 1062 1256 1120 1955 1623 1013 1954
 1177 

In [4]:
client1=pd.DataFrame()

client1=pd.concat([client1,df[df['SMID'] ==id_list[0]]])
client1.head(4000)
# len(client1)


,SMID,TimeID,Energy
0,1392,19503,0.140
1,1392,19504,0.138
2,1392,19505,0.140
3,1392,19506,0.145
4,1392,19507,0.145
...,...,...,...
3671083,1392,27744,0.883
3671084,1392,27745,0.687
3671085,1392,27746,0.662
3671086,1392,27747,0.666


In [5]:
client1['SMID'].unique()

array([1392])

In [6]:
client1.head(5000)

,SMID,TimeID,Energy
0,1392,19503,0.140
1,1392,19504,0.138
2,1392,19505,0.140
3,1392,19506,0.145
4,1392,19507,0.145
...,...,...,...
3671083,1392,27744,0.883
3671084,1392,27745,0.687
3671085,1392,27746,0.662
3671086,1392,27747,0.666


In [7]:
client1=client1[['TimeID','Energy']]
client1

,TimeID,Energy
0,19503,0.140
1,19504,0.138
2,19505,0.140
3,19506,0.145
4,19507,0.145
...,...,...
3671083,27744,0.883
3671084,27745,0.687
3671085,27746,0.662
3671086,27747,0.666


In [8]:
from datetime import datetime
from datetime import timedelta
def extract_datetime(time_id):
    day_no= str(time_id)[0:3]
    day_no=int(day_no)
    slot_no=str(time_id)[3:5]
    slot_no=int(slot_no)
    def_date=datetime(year=2009,month=1,day=1)
    day_delta=timedelta(days=day_no-1)
    month=def_date+day_delta
    time_delta=timedelta(minutes=(slot_no-1)*30)
    month=month+time_delta
    return month.strftime("%m/%d/%Y %H:%M:%S")
    # month.strftime('%d/%m/%Y')
extract_datetime(19503)

'07/14/2009 01:00:00'

In [9]:
client1['Time']=client1['TimeID'].apply(extract_datetime)

In [10]:
client1.head()

,TimeID,Energy,Time
0,19503,0.140,07/14/2009 01:00:00
1,19504,0.138,07/14/2009 01:30:00
2,19505,0.140,07/14/2009 02:00:00
3,19506,0.145,07/14/2009 02:30:00
4,19507,0.145,07/14/2009 03:00:00


In [11]:
client1=client1[['Time','Energy']]

In [12]:
client1.head()

,Time,Energy
0,07/14/2009 01:00:00,0.140
1,07/14/2009 01:30:00,0.138
2,07/14/2009 02:00:00,0.140
3,07/14/2009 02:30:00,0.145
4,07/14/2009 03:00:00,0.145


In [13]:
client1.set_index('Time',inplace=True)
client1.index = pd.to_datetime(client1.index)
 

In [14]:
# idx = pd.date_range("2017-01-01", periods=2)
client1.index=client1.index.to_period(freq='30min')

client1.head()

,Energy
Time,
2009-07-14 00:00,0.157
2009-07-14 00:30,0.144
2009-07-14 01:00,0.140
2009-07-14 01:30,0.138
2009-07-14 02:00,0.140


In [15]:
#Training with first 100 time slots for client 1392
from statsmodels.tsa.arima.model import ARIMA
model = ARIMA(client1[0:100], order=(5,1,0))
fit_model=model.fit()
print(fit_model.summary())
fit_model.resid

                               SARIMAX Results                                
Dep. Variable:                 Energy   No. Observations:                  100
Model:                 ARIMA(5, 1, 0)   Log Likelihood                 -13.670
Date:                Sun, 24 Dec 2023   AIC                             39.341
Time:                        23:40:48   BIC                             54.911
Sample:                    07-14-2009   HQIC                            45.641
                         - 07-16-2009                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.7113      0.059    -12.079      0.000      -0.827      -0.596
ar.L2         -0.4284      0.071     -6.017      0.000      -0.568      -0.289
ar.L3         -0.3534      0.115     -3.086      0.0

Time
2009-07-14 00:00    0.157000
2009-07-14 00:30   -0.013000
2009-07-14 01:00   -0.010562
2009-07-14 01:30   -0.007260
2009-07-14 02:00   -0.002927
                      ...   
2009-07-15 23:30   -0.279986
2009-07-16 00:00   -0.224797
2009-07-16 00:30   -0.202641
2009-07-16 01:00   -0.047124
2009-07-16 01:30   -0.048427
Freq: 30T, Length: 100, dtype: float64

In [16]:
client1.head(20)

,Energy
Time,
2009-07-14 00:00,0.157
2009-07-14 00:30,0.144
2009-07-14 01:00,0.140
2009-07-14 01:30,0.138
2009-07-14 02:00,0.140
2009-07-14 02:30,0.145
2009-07-14 03:00,0.145
2009-07-14 03:30,0.144
2009-07-14 04:00,0.141


In [17]:
print(len(client1))
client1.iloc[-1]

3984


Energy    0.457
Name: 2009-10-04 23:30, dtype: float64

In [18]:
fit_model.forecast(steps=2)

2009-07-16 02:00    0.160415
2009-07-16 02:30    0.164360
Freq: 30T, Name: predicted_mean, dtype: float64

In [19]:
print(client1.iloc[99])
print(client1.iloc[100])
print(client1.iloc[101])


Energy    0.135
Name: 2009-07-16 01:30, dtype: float64
Energy    0.144
Name: 2009-07-16 02:00, dtype: float64
Energy    0.152
Name: 2009-07-16 02:30, dtype: float64


In [20]:
from math import sqrt

from sklearn.metrics import mean_squared_error


X= client1.values
size = 3000
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()
# walk-forward validation
for t in range(len(test)):
 model = ARIMA(history, order=(5,1,0))
 model_fit = model.fit()
 output = model_fit.forecast()
 yhat = output[0]
 predictions.append(yhat)
 obs = test[t]
 history.append(obs)
 print('predicted=%f, expected=%f' % (yhat, obs))
# evaluate forecasts
rmse = sqrt(mean_squared_error(test, predictions))

/var/folders/xx/0p3yl_7x1xxgr4qztg9qrr8w0000gn/T/ipykernel_26941/2740836441.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('predicted=%f, expected=%f' % (yhat, obs))


predicted=0.136878, expected=0.136000
predicted=0.136757, expected=0.134000
predicted=0.135560, expected=0.134000
predicted=0.134948, expected=0.134000
predicted=0.134596, expected=0.137000
predicted=0.135848, expected=0.226000
predicted=0.177199, expected=0.140000
predicted=0.148833, expected=0.138000
predicted=0.147139, expected=0.137000
predicted=0.143130, expected=0.133000
predicted=0.144536, expected=0.174000
predicted=0.165366, expected=0.229000
predicted=0.184311, expected=0.239000
predicted=0.199915, expected=0.380000
predicted=0.274848, expected=0.323000
predicted=0.274971, expected=0.345000
predicted=0.305234, expected=2.288000
predicted=1.219107, expected=0.398000
predicted=0.610159, expected=0.839000
predicted=0.799889, expected=0.773000
predicted=0.740527, expected=0.548000
predicted=0.753897, expected=0.603000
predicted=0.827070, expected=0.258000
predicted=0.448977, expected=0.181000
predicted=0.407827, expected=0.181000
predicted=0.330276, expected=0.172000
predicted=0.

### Model Performance:

Root Mean Square Error:

In [21]:
# lower the value of rmse the better it is
print(rmse)

0.36918399996206736


R Square Coefficient:

In [22]:
# Should be between 0.5 and 0.99
from sklearn.metrics import r2_score
print(r2_score(test,predictions))


0.25677749500645153


In [23]:
client1.to_csv('Client1.csv')